In [1]:
from __future__ import print_function, division
from sympy.core import (
    S, Basic, Expr, I, Integer, Add, Mul, Dummy, Tuple
)

from sympy.core.mul import _keep_coeff
from sympy.core.symbol import Symbol
from sympy.core.basic import preorder_traversal
from sympy.core.relational import Relational
from sympy.core.sympify import sympify
from sympy.core.decorators import _sympifyit
from sympy.core.function import Derivative

from sympy.logic.boolalg import BooleanAtom

from sympy.polys.polyclasses import DMP

from sympy.polys.polyutils import (
    basic_from_dict,
    _sort_gens,
    _unify_gens,
    _dict_reorder,
    _dict_from_expr,
    _parallel_dict_from_expr,
)

from sympy.polys.rationaltools import together
from sympy.polys.rootisolation import dup_isolate_real_roots_list
from sympy.polys.groebnertools import groebner as _groebner
from sympy.polys.fglmtools import matrix_fglm
from sympy.polys.monomials import Monomial
from sympy.polys.orderings import monomial_key

from sympy.polys.polyerrors import (
    OperationNotSupported, DomainError,
    CoercionFailed, UnificationFailed,
    GeneratorsNeeded, PolynomialError,
    MultivariatePolynomialError,
    ExactQuotientFailed,
    PolificationFailed,
    ComputationFailed,
    GeneratorsError,
)

from sympy.utilities import group, sift, public, filldedent

import sympy.polys
import mpmath
from mpmath.libmp.libhyper import NoConvergence

from sympy.polys.domains import FF, QQ, ZZ
from sympy.polys.constructor import construct_domain

from sympy.polys import polyoptions as options

from sympy.core.compatibility import iterable, range, ordered
from sympy.polys.polytools import *


from sympy.polys.monomials import monomial_mul, monomial_lcm, monomial_divides, term_div
from sympy.polys.orderings import lex
from sympy.polys.polyerrors import DomainError
from sympy.polys.polyconfig import query
from sympy.core.symbol import Dummy
from sympy.core.compatibility import range
from sympy.polys.groebnertools import *

@public
def groebnerbasis(iterative, F, n, *gens, **args):
    return GroebnerBasis(iterative, F, n, *gens, **args)


@public
class GroebnerBasis(Basic):
    """Represents a reduced Groebner basis. """

    def __new__(cls, iterative, F, n, *gens, **args):
        """Compute a reduced Groebner basis for a system of polynomials. """
        options.allowed_flags(args, ['polys', 'method'])
        try:
            polys, opt = parallel_poly_from_expr(F, *gens, **args)
        except PolificationFailed as exc:
            raise ComputationFailed('groebner', len(F), exc)

        from sympy.polys.rings import PolyRing
        ring = PolyRing(opt.gens, opt.domain, opt.order)

        polys = [ring.from_dict(poly.rep.to_dict()) for poly in polys if poly]
        if iterative:
            G = iter_groebner(polys,n,ring, method = opt.method)#Assumes last element is the "new" polynomial
        else:
            G = _groebner(polys, ring, method=opt.method)
        if G == 'Timeout':
            return 'Timeout'
        G = [Poly._from_dict(g, opt) for g in G]

        return cls._new(G, opt)

    @classmethod
    def _new(cls, basis, options):
        obj = Basic.__new__(cls)

        obj._basis = tuple(basis)
        obj._options = options

        return obj

    @property
    def args(self):
        return (Tuple(*self._basis), Tuple(*self._options.gens))

    @property
    def exprs(self):
        return [poly.as_expr() for poly in self._basis]

    @property
    def polys(self):
        return list(self._basis)

    @property
    def gens(self):
        return self._options.gens

    @property
    def domain(self):
        return self._options.domain

    @property
    def order(self):
        return self._options.order

    def __len__(self):
        return len(self._basis)

    def __iter__(self):
        if self._options.polys:
            return iter(self.polys)
        else:
            return iter(self.exprs)

    def __getitem__(self, item):
        if self._options.polys:
            basis = self.polys
        else:
            basis = self.exprs

        return basis[item]

    def __hash__(self):
        return hash((self._basis, tuple(self._options.items())))

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self._basis == other._basis and self._options == other._options
        elif iterable(other):
            return self.polys == list(other) or self.exprs == list(other)
        else:
            return False

    def __ne__(self, other):
        return not self == other

    @property
    def is_zero_dimensional(self):
        """
        Checks if the ideal generated by a Groebner basis is zero-dimensional.

        The algorithm checks if the set of monomials not divisible by the
        leading monomial of any element of ``F`` is bounded.

        References
        ==========

        David A. Cox, John B. Little, Donal O'Shea. Ideals, Varieties and
        Algorithms, 3rd edition, p. 230

        """
        def single_var(monomial):
            return sum(map(bool, monomial)) == 1

        exponents = Monomial([0]*len(self.gens))
        order = self._options.order

        for poly in self.polys:
            monomial = poly.LM(order=order)

            if single_var(monomial):
                exponents *= monomial

        # If any element of the exponents vector is zero, then there's
        # a variable for which there's no degree bound and the ideal
        # generated by this Groebner basis isn't zero-dimensional.
        return all(exponents)

    def fglm(self, order):
        """
        Convert a Groebner basis from one ordering to another.

        The FGLM algorithm converts reduced Groebner bases of zero-dimensional
        ideals from one ordering to another. This method is often used when it
        is infeasible to compute a Groebner basis with respect to a particular
        ordering directly.

        Examples
        ========

        >>> from sympy.abc import x, y
        >>> from sympy import groebner

        >>> F = [x**2 - 3*y - x + 1, y**2 - 2*x + y - 1]
        >>> G = groebner(F, x, y, order='grlex')

        >>> list(G.fglm('lex'))
        [2*x - y**2 - y + 1, y**4 + 2*y**3 - 3*y**2 - 16*y + 7]
        >>> list(groebner(F, x, y, order='lex'))
        [2*x - y**2 - y + 1, y**4 + 2*y**3 - 3*y**2 - 16*y + 7]

        References
        ==========

        J.C. Faugere, P. Gianni, D. Lazard, T. Mora (1994). Efficient
        Computation of Zero-dimensional Groebner Bases by Change of
        Ordering

        """
        opt = self._options

        src_order = opt.order
        dst_order = monomial_key(order)

        if src_order == dst_order:
            return self

        if not self.is_zero_dimensional:
            raise NotImplementedError("can't convert Groebner bases of ideals with positive dimension")

        polys = list(self._basis)
        domain = opt.domain

        opt = opt.clone(dict(
            domain=domain.get_field(),
            order=dst_order,
        ))

        from sympy.polys.rings import xring
        _ring, _ = xring(opt.gens, opt.domain, src_order)

        for i, poly in enumerate(polys):
            poly = poly.set_domain(opt.domain).rep.to_dict()
            polys[i] = _ring.from_dict(poly)

        G = matrix_fglm(polys, _ring, dst_order)
        G = [Poly._from_dict(dict(g), opt) for g in G]

        if not domain.is_Field:
            G = [g.clear_denoms(convert=True)[1] for g in G]
            opt.domain = domain

        return self._new(G, opt)

    def reduce(self, expr, auto=True):
        """
        Reduces a polynomial modulo a Groebner basis.

        Given a polynomial ``f`` and a set of polynomials ``G = (g_1, ..., g_n)``,
        computes a set of quotients ``q = (q_1, ..., q_n)`` and the remainder ``r``
        such that ``f = q_1*f_1 + ... + q_n*f_n + r``, where ``r`` vanishes or ``r``
        is a completely reduced polynomial with respect to ``G``.

        Examples
        ========

        >>> from sympy import groebner, expand
        >>> from sympy.abc import x, y

        >>> f = 2*x**4 - x**2 + y**3 + y**2
        >>> G = groebner([x**3 - x, y**3 - y])

        >>> G.reduce(f)
        ([2*x, 1], x**2 + y**2 + y)
        >>> Q, r = _

        >>> expand(sum(q*g for q, g in zip(Q, G)) + r)
        2*x**4 - x**2 + y**3 + y**2
        >>> _ == f
        True

        """
        poly = Poly._from_expr(expr, self._options)
        polys = [poly] + list(self._basis)

        opt = self._options
        domain = opt.domain

        retract = False

        if auto and domain.is_Ring and not domain.is_Field:
            opt = opt.clone(dict(domain=domain.get_field()))
            retract = True

        from sympy.polys.rings import xring
        _ring, _ = xring(opt.gens, opt.domain, opt.order)

        for i, poly in enumerate(polys):
            poly = poly.set_domain(opt.domain).rep.to_dict()
            polys[i] = _ring.from_dict(poly)

        Q, r = polys[0].div(polys[1:])

        Q = [Poly._from_dict(dict(q), opt) for q in Q]
        r = Poly._from_dict(dict(r), opt)

        if retract:
            try:
                _Q, _r = [q.to_ring() for q in Q], r.to_ring()
            except CoercionFailed:
                pass
            else:
                Q, r = _Q, _r

        if not opt.polys:
            return [q.as_expr() for q in Q], r.as_expr()
        else:
            return Q, r


    def contains(self, poly):
        """
        Check if ``poly`` belongs the ideal generated by ``self``.

        Examples
        ========

        >>> from sympy import groebner
        >>> from sympy.abc import x, y

        >>> f = 2*x**3 + y**3 + 3*y
        >>> G = groebner([x**2 + y**2 - 1, x*y - 2])

        >>> G.contains(f)
        True
        >>> G.contains(f + 1)
        False

        """
        return self.reduce(poly)[1] == 0

def iter_groebner(seq,n, ring, method=None):
    """
    Computes Groebner basis for a set of polynomials in `K[X]`.

    Wrapper around the (default) improved Buchberger and the other algorithms
    for computing Groebner bases. The choice of algorithm can be changed via
    ``method`` argument or :func:`setup` from :mod:`sympy.polys.polyconfig`,
    where ``method`` can be either ``buchberger`` or ``f5b``.

    """

    domain, orig = ring.domain, None
    if not domain.is_Field or not domain.has_assoc_Field:
        try:
            orig, ring = ring, ring.clone(domain=domain.get_field())
        except DomainError:
            raise DomainError("can't compute a Groebner basis over %s" % domain)
        else:
            seq = [ s.set_ring(ring) for s in seq ]

    G = _incr_buch(seq,n, ring)
    if G == 'Timeout':
        return 'Timeout'
    if orig is not None:
        G = [ g.clear_denoms()[1].set_ring(orig) for g in G ]

    return G

def _incr_buch(f,n, ring):
    """
    Incremental Computation of a reduced Grobner basis, given that f[:-1] is a reduced Grobner basis
    and f[-1] is a new polynomial to add into the basis.
    New polynomial f[-1] is assumed to be reduced by the grobner basis f[:-1] already.

    """
    order = ring.order
    domain = ring.domain

    monomial_mul = ring.monomial_mul
    monomial_div = ring.monomial_div
    monomial_lcm = ring.monomial_lcm

    def select(P):
        # normal selection strategy
        # select the pair with minimum LCM(LM(f), LM(g))
        pr = min(P, key=lambda pair: order(monomial_lcm(f[pair[0]].LM, f[pair[1]].LM)))
        return pr

    def normal(g, J):
        h = g.rem([ f[j] for j in J ])

        if not h:
            return None
        else:
            h = h.monic()

            if not h in I:
                I[h] = len(f)
                f.append(h)

            return h.LM, I[h]

    def update(G, B, ih):
        # update G using the set of critical pairs B and h
        # [BW] page 230
        h = f[ih]
        mh = h.LM

        # filter new pairs (h, g), g in G
        C = G.copy()
        D = set()

        while C:
            # select a pair (h, g) by popping an element from C
            ig = C.pop()
            g = f[ig]
            mg = g.LM
            LCMhg = monomial_lcm(mh, mg)

            def lcm_divides(ip):
                # LCM(LM(h), LM(p)) divides LCM(LM(h), LM(g))
                m = monomial_lcm(mh, f[ip].LM)
                return monomial_div(LCMhg, m)

            # HT(h) and HT(g) disjoint: mh*mg == LCMhg
            if monomial_mul(mh, mg) == LCMhg or (
                not any(lcm_divides(ipx) for ipx in C) and
                    not any(lcm_divides(pr[1]) for pr in D)):
                D.add((ih, ig))

        E = set()

        while D:
            # select h, g from D (h the same as above)
            ih, ig = D.pop()
            mg = f[ig].LM
            LCMhg = monomial_lcm(mh, mg)

            if not monomial_mul(mh, mg) == LCMhg:
                E.add((ih, ig))

        # filter old pairs
        B_new = set()

        while B:
            # select g1, g2 from B (-> CP)
            ig1, ig2 = B.pop()
            mg1 = f[ig1].LM
            mg2 = f[ig2].LM
            LCM12 = monomial_lcm(mg1, mg2)

            # if HT(h) does not divide lcm(HT(g1), HT(g2))
            if not monomial_div(LCM12, mh) or \
                monomial_lcm(mg1, mh) == LCM12 or \
                    monomial_lcm(mg2, mh) == LCM12:
                B_new.add((ig1, ig2))

        B_new |= E

        # filter polynomials
        G_new = set()

        while G:
            ig = G.pop()
            mg = f[ig].LM

            if not monomial_div(mg, mh):
                G_new.add(ig)

        G_new.add(ih)

        return G_new, B_new
        # end of update ################################
    start = time.time()
    if not f:
        return []
    f1 = [func for func in f[:-n]]
    for p in f[-n:]:
        r = p.rem(f1)
        if r != 0:
            f1.append(r)
    f = f1
    I = {} # ip = I[p]; p = f[ip]
    F = set()
    G = set()         # set of indices of intermediate would-be Groebner basis
    CP = set()        # set of pairs of indices of critical pairs

    for i, h in enumerate(f):
        I[h] = i #Setup polynomial-index dictionary
        if i >= len(f)-n:
            F.add(i)
        else:
            G.add(i)
            
    #####################################
    # algorithm GROEBNERNEWS2 in [BW] page 232
    while F:
        # select p with minimum monomial according to the monomial ordering
        h = min([f[x] for x in F], key=lambda f: order(f.LM))
        ih = I[h]
        F.remove(ih)
        G, CP = update(G, CP, ih)

    # count the number of critical pairs which reduce to zero
    reductions_to_zero = 0

    while CP:
        elapsed = time.time() - start
        if elapsed >= 300:
            return 'Timeout'
        ig1, ig2 = select(CP)
        CP.remove((ig1, ig2))
        
        h = spoly(f[ig1], f[ig2], ring)
        # ordering divisors is on average more efficient [Cox] page 111
        G1 = sorted(G, key=lambda g: order(f[g].LM))
        ht = normal(h, G1)

        if ht:
            G, CP = update(G, CP, ht[1])
        else:
            reductions_to_zero += 1

    ######################################
    # now G is a Groebner basis; reduce it
    Gr = set()

    for ig in G:
        ht = normal(f[ig], G - set([ig]))

        if ht:
            Gr.add(ht[1])

    Gr = [f[ig] for ig in Gr]

    # order according to the monomial ordering
    Gr = sorted(Gr, key=lambda f: order(f.LM), reverse=True)

    return Gr

In [2]:
import numpy as np
from math import floor
import time
from itertools import product
from itertools import combinations
from sympy import Matrix,var

In [3]:
#Useful functions for both Hypercubes and general Hamming graphs
def read_sets(filename):
    #Read in the sets of a file
    results = []
    with open(filename,'r') as f:
        for line in f.read().splitlines():
            results.append(line.split(','))
    return(results)

def make_linearEqns(A,z):
    #Takes matrix A and variable list z and converts it into a list of linear equations
    zmat = Matrix(z)
    A = Matrix(A)
    A = A.rref()[0]
    lin_fcns = A*zmat
    lin_fcns = [f for f in lin_fcns if f != 0]
    lin_fcns = lin_fcns[::-1]
    return(list(lin_fcns))

In [4]:
def OneHot(R,k,a,alphabet = None):
    #Converts list of strings to list of one-hot encodings
    if alphabet == None:
        temp = [str(i) for i in range(a)]
        alphabet = ''.join(temp)
    encodings = []
    for r in R:
        encoding = np.zeros((a,k))
        for i in range(k):
            for j,letter in enumerate(alphabet):
                if r[i] == letter:
                    encoding[j,i] = 1
        encodings.append(encoding)
    return(encodings)

def create_groebner(k,a):
    #Constructs known groebner basis based on pattern for P and creates fs and variable vector.
    variable_string = ''
    num_digits = len(str(k*a))
    for i in range(0,k*a):
        count = str(i+1).zfill(num_digits)
        variable = ',z'+count
        variable_string = variable_string +variable
    variable_string = variable_string[1:]
    z = var(variable_string)
    z = tuple(z)
    G0 = []
    for i in range(1,a):
        G0.append(z[i]**3-z[i])
    pairs = combinations(range(1,a),2)
    for pair in pairs:
        i = pair[0]
        j = pair[1]
        G0.append(z[i]**2*z[j]+z[i]*z[j]**2)
    triplets = combinations(range(1,a),3)
    for triple in triplets:
        i = triple[0]
        j = triple[1]
        l = triple[2]
        G0.append(z[i]*z[j]*z[l])
    G = [g for g in list(G0)]
    for i in range(1,k):
        replacements = [(z[j],z[a*i+j]) for j in range(a)]
        for g in list(G0):
            G.append(g.subs(replacements))
    squares = [zi**2 for zi in z]
    sum_squares = sum(squares)
    fs = [sum_squares-2*i for i in range(1,k+1)]
    return(G,fs,z)

def make_matrix(R,k,a):
    #Converts list of one-hot encodings to the linear system
    temp = [r.flatten('F') for r in R]
    for i in range(k):
        added_row = np.zeros(k*a)
        for j in range(a):
            #This is the 2nd condition where sum(zi)_i*a+1^i*a+a = 0
            added_row[i*a+j] = 1
        temp.append(list(added_row))
    return(np.array(temp,dtype = int))

def check_resolving(R,k,a,alphabet = None):
    (G,fs,z) = create_groebner(k,a)
    OH_encodedR = OneHot(R,k,a,alphabet = alphabet)
    A = make_matrix(OH_encodedR,k,a)
    lin_fcns = make_linearEqns(A,z)
    n = len(lin_fcns)
    #print("Made Linear Functions")
    G = groebnerbasis(True,G+lin_fcns,n,z, order = 'lex')
    if G == 'Timeout':
        return 'Timeout'
    #print("Finished Linear Functions")
    for i,fi in enumerate(fs):
        Gi = groebnerbasis(True,list(G)+[fi],1,z, order = 'lex')
        if Gi == 'Timeout':
            return 'Timeout'
        if list(Gi) != [1]:
            return False
    return True


In [5]:
def hypercube_matrix(R):
    A = np.zeros((len(R),len(R[0])))
    for i,r in enumerate(R):
        for j,letter in enumerate(r):
            if letter == '0':
                A[i,j] = -1
            else:
                A[i,j] = int(letter)
    return(A)

def hypercube_polys(k):
    variable_string = ''
    num_digits = len(str(k))
    for i in range(0,k):
        count = str(i+1).zfill(num_digits)
        variable = ',z'+count
        variable_string = variable_string +variable
    variable_string = variable_string[1:]
    z = var(variable_string)
    P = []
    f = 0
    for i in range(k):
        P.append(z[i]*(z[i]-1)*(z[i]+1))
        f = f + z[i]**2
    fs = []
    for i in range(floor(k/2)):
        fi = f-2*(i+1)
        fs.append(fi)
    return(P,fs,z)

def check_hcube_resolving(R,k):
    #Create matrix A from R
    A = hypercube_matrix(R)
    #Get pre-computed Groebner basis and variables for H_k,2
    G,fs,z = hypercube_polys(k)
    #Get linear functions from A matrix
    lin_fcns = make_linearEqns(A,z)
    n = len(lin_fcns)
    #Get Grobner basis of P and linear functions
    G = groebnerbasis(True,G+lin_fcns,n,order = 'lex')
    for i,fi in enumerate(fs):
        #Compute Grobner basis of G+fi
        Gi = groebnerbasis(True,list(G)+[fi],1,order = 'lex')
        #Solutions iff Gi neq 1, if Gi neq 1 then R is not resolving
        if list(Gi) != [1]:
            return False
    return True
    

In [7]:
def dist_mat(nodes,R):
    distMat = np.zeros((len(nodes), len(R)))
    for i,n in enumerate(nodes):
        for j,r in enumerate(R):
            distMat[i,j] = sum(1 for c1, c2 in zip(n, r) if c1 != c2)
    return(distMat)

def brute_force_resolve(R,k,a, alphabet = None, timeout = None):
    start = time.time()
    if alphabet == None:
        temp = [str(i) for i in range(a)]
        alphabet = ''.join(temp)
    H = [''.join(x) for x in product(alphabet,repeat = k)]
    distMat = dist_mat(H,R)
    n = len(H)
    for i in range(n-1):
        for j in range(i+1,n):
            if timeout != None:
                elapsed = time.time() - start
                if elapsed >= timeout:
                    return 'Timeout'
            if all(distMat[i,:] == distMat[j,:]):
                return(False)
    return(True)

def hammingDist(a, b):
    return sum(1 for (x,y) in zip(a,b) if x!=y)

def checkResolvingHamming(R, k, a, alphabet = None, timeout = None):
    start = time.time()
    if alphabet == None:
        temp = [str(i) for i in range(a)]
        alphabet = ''.join(temp)
    tags = {}
    for seq in product(alphabet, repeat=k):
        end = time.time()
        if end-start >= timeout:
            return 'Timeout'
        tag = ';'.join(map(str, [hammingDist(seq, r) for r in R]))
        if tag in tags: return False
        tags[tag] = 1
    return True

In [12]:
from sys import *
import os
class RunTimeData:
    data = []
    
    def __init__(self):
        self.groeb_resDict = {}
        self.groeb_noresDict = {}
        self.brute_resDict = {}
        self.brute_noresDict = {}
    
    def load_data(self,inFile):
        with open(inFile, 'r') as f:
            f.readline() #header
            for line in f:
                l = line.strip().split('\t')
                k = int(l[0])
                a = int(l[1])
                R = l[2].split(',')
                isResolving = l[3] == "True"
                if (k,a,R,isResolving) not in self.data:
                    self.data.append((k, a, R, isResolving))

    def prelim_groebner(self,threshold, verbose = False):
        begin = time.time()
        todo_indices = []
        for i,data in enumerate(self.data):
            k = data[0]
            a = data[1]
            if a*k <= threshold:
                todo_indices.append(i)
        num_todo = len(todo_indices)
        if verbose:
            print("Found {} examples to run".format(num_todo))
        for i,ind in enumerate(todo_indices):
            data = self.data[ind]
            k = data[0]
            a = data[1]
            R = data[2]
            isResolving = data[3]
            size = len(R)
            elapsed = time.time()-begin
            if verbose:
                write_string = "(k,a) = ({},{}) Starting example {}/{}, time_elapsed = {:.2f} \r".format(k,a,i+1,num_todo,elapsed)
                stdout.write(write_string)
            if a == 2:
                if k == 1:
                    start = time.time()
                    resolved = True
                    end = time.time()
                    time_taken = end-start
                else:
                    start = time.time()
                    resolved = check_hcube_resolving(R,k)
                    end = time.time()
                    time_taken = end-start
            else:
                start = time.time()
                resolved = check_resolving(R,k,a)
                end = time.time()
                if resolved == 'Timeout':
                    time_taken = 'Timeout'
                else:
                    time_taken = end-start
            
            if isResolving:
                key = (k,a)
                if key in self.groeb_resDict.keys():
                    self.groeb_resDict[key].append((time_taken,size))
                else:
                    self.groeb_resDict[key] = [(time_taken,size)]
            else:
                key = (k,a)
                if key in self.groeb_noresDict.keys():
                    self.groeb_noresDict[key].append((time_taken,size))
                else:
                    self.groeb_noresDict[key] = [(time_taken,size)]
            if verbose:
                elapsed = time.time()-begin
                write_string = "(k,a) = ({},{}) Finished example {}/{}, time elapsed = {:.2f} \r".format(k,a,i+1,num_todo,elapsed)
                stdout.write(write_string)
                stdout.flush() 
                
    def prelim_bruteforce(self,threshold,verbose = False, timeout = 300):
        begin = time.time()
        todo_indices = []
        for i,data in enumerate(self.data):
            k = data[0]
            a = data[1]
            if a*k <= threshold:
                todo_indices.append(i)
        num_todo = len(todo_indices)
        if verbose:
            print("Found {} examples to run".format(num_todo))
        for i,ind in enumerate(todo_indices):
            data = self.data[ind]
            k = data[0]
            a = data[1]
            R = data[2]
            isResolving = data[3]
            size = len(R)
            elapsed = time.time()-begin
            if verbose:
                write_string = "Starting example {}/{}, time_elapsed = {:.2f} \r".format(i+1,num_todo,elapsed)
                stdout.write(write_string)
            start = time.time()
            resolved = checkResolvingHamming(R,k,a, timeout = timeout)
            end = time.time()
            if resolved:
                if not isResolving:
                    print("Error in checking resolving")
            if not resolved:
                if isResolving:
                    print("Error in checking resolving")
            if resolved == 'Timeout':
                time_taken = 'TImeout'
            else:
                time_taken = end-start
            if isResolving:
                key = (k,a)
                if key in self.brute_resDict.keys():
                    self.brute_resDict[key].append((time_taken,size))
                else:
                    self.brute_resDict[key] = [(time_taken,size)]
            else:
                key = (k,a)
                if key in self.brute_noresDict.keys():
                    self.brute_noresDict[key].append((time_taken,size))
                else:
                    self.brute_noresDict[key] = [(time_taken,size)]
            if verbose:
                elapsed = time.time()-begin
                write_string = "Finished example {}/{}, time elapsed = {:.2f} \r".format(i+1,num_todo,elapsed)
                stdout.write(write_string)
                stdout.flush() 

In [13]:
experiment_data = RunTimeData()
experiment_data.load_data('res_set_data.tsv')
experiment_data.load_data('res_set_data_2.tsv')

In [82]:
experiment_data.prelim_groebner(25,verbose = True)

Found 4359 examples to run


In [14]:
experiment_data.prelim_bruteforce(25,verbose = True)

Found 4359 examples to run


In [87]:
dictionary = experiment_data.groeb_resDict
print(dictionary)

{(10, 2): [(0.10024738311767578, 7), (0.04685497283935547, 7), (0.03125405311584473, 7), (0.031228303909301758, 7), (0.0313107967376709, 7), (0.03341317176818848, 7), (0.031247615814208984, 7), (0.03128790855407715, 7), (0.031238555908203125, 7), (0.031243562698364258, 7), (0.031246662139892578, 7), (0.04685211181640625, 7), (0.04686713218688965, 7), (0.03128242492675781, 7), (0.031229496002197266, 7), (0.031212568283081055, 7), (0.03128838539123535, 7), (0.031239986419677734, 7), (0.031231403350830078, 7), (0.031243324279785156, 7), (0.03126239776611328, 7), (0.03123331069946289, 7), (0.12492680549621582, 7), (0.06253409385681152, 7), (0.015604257583618164, 7), (0.046872615814208984, 7), (0.031247615814208984, 7), (0.05184507369995117, 7), (0.04691457748413086, 7), (0.03119516372680664, 7), (0.046898603439331055, 7), (0.031208276748657227, 7), (0.03127551078796387, 7), (0.03334403038024902, 7), (0.04687762260437012, 7), (0.046853065490722656, 7), (0.031240463256835938, 7), (0.03125500

In [89]:
import pickle
with open('groeb_res.dict','wb') as f:
    dictionary = experiment_data.groeb_resDict
    pickle.dump(dictionary,f)

In [90]:
with open('groeb_nores.dict','wb') as f:
    dictionary = experiment_data.groeb_noresDict
    pickle.dump(dictionary,f)
    
with open('brute_res.dict','wb') as f:
    dictionary = experiment_data.brute_resDict
    pickle.dump(dictionary,f)
    
with open('brute_nores.dict','wb') as f:
    dictionary = experiment_data.brute_noresDict
    pickle.dump(dictionary,f)

In [63]:
sets = read_sets('Test_Sets/H_2_8.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_hcube_resolving(R,8)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.04539847373962402
0.0646209716796875
0.04887247085571289
0.06572794914245605
0.05683088302612305
0.09498882293701172
0.04929184913635254
0.049120187759399414
Set 1 is not resolving
Set 2 is not resolving
Set 3 is resolving
Set 4 is resolving
Set 5 is not resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 0.47 seconds
Average time: 0.06 seconds


In [64]:
sets = read_sets('Test_Sets/H_2_8.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = brute_force_resolve(R,8,2)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.006655693054199219
0.006608724594116211
0.08054947853088379
0.08321189880371094
0.0
0.08335185050964355
0.09584331512451172
0.005370140075683594
Set 1 is not resolving
Set 2 is not resolving
Set 3 is resolving
Set 4 is resolving
Set 5 is not resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 0.36 seconds
Average time: 0.05 seconds


In [65]:
sets = read_sets('Test_Sets/H_2_12.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_hcube_resolving(R,12)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.10807585716247559
0.06663942337036133
0.06670069694519043
0.11447763442993164
0.10610842704772949
0.13337349891662598
0.08268332481384277
0.08605337142944336
Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is not resolving
Set 8 is not resolving
Total Time: 0.76 seconds
Average time: 0.10 seconds


In [80]:
sets = read_sets('Test_Sets/H_2_12.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = brute_force_resolve(R,12,2)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

19.561630487442017
0.14059042930603027
0.1763772964477539
19.33766508102417
19.096622943878174
19.026614665985107
0.45575594902038574
0.15189862251281738
Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is not resolving
Set 8 is not resolving
Total Time: 77.95 seconds
Average time: 9.74 seconds


In [68]:
sets = read_sets('Test_Sets/H_3_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,5,3)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.15196990966796875
0.14442229270935059
0.2472395896911621
0.20987415313720703
0.15610527992248535
0.25594544410705566
0.17330384254455566
0.16269946098327637
Set 1 is resolving
Set 2 is resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is not resolving
Set 6 is not resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 1.50 seconds
Average time: 0.19 seconds


In [69]:
sets = read_sets('Test_Sets/H_3_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = brute_force_resolve(R,5,3)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.08566737174987793
0.08269119262695312
0.07848930358886719
0.021083593368530273
0.032106876373291016
0.0009984970092773438
0.06749844551086426
0.015045404434204102
Set 1 is resolving
Set 2 is resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is not resolving
Set 6 is not resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 0.38 seconds
Average time: 0.05 seconds


In [70]:
sets = read_sets('Test_Sets/H_3_6.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,6,3)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

1.0516488552093506
1.6908600330352783
6.4284820556640625
3.676518440246582
1.632089376449585
2.4064900875091553
0.19938397407531738
0.4181351661682129
Set 1 is resolving
Set 2 is not resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is not resolving
Set 7 is not resolving
Set 8 is resolving
Total Time: 17.50 seconds
Average time: 2.19 seconds


In [71]:
sets = read_sets('Test_Sets/H_3_6.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = brute_force_resolve(R,6,3)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.698009729385376
0.016595125198364258
0.6915011405944824
0.03231978416442871
0.6566340923309326
0.08814096450805664
0.02576732635498047
0.6960139274597168
Set 1 is resolving
Set 2 is not resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is not resolving
Set 7 is not resolving
Set 8 is resolving
Total Time: 2.90 seconds
Average time: 0.36 seconds


In [72]:
sets = read_sets('Test_Sets/H_3_7.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,7,3)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

2.4470438957214355
0.8273015022277832
0.8475520610809326
1.2255289554595947
1.4212887287139893
2.6958272457122803
6.971927881240845
31.25673770904541
Set 1 is resolving
Set 2 is resolving
Set 3 is not resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is not resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 47.69 seconds
Average time: 5.96 seconds


In [73]:
sets = read_sets('Test_Sets/H_3_7.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = brute_force_resolve(R,7,3)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

7.395327091217041
6.169711589813232
0.15079164505004883
0.11328673362731934
6.195536375045776
1.735015630722046
6.644796848297119
0.07799148559570312
Set 1 is resolving
Set 2 is resolving
Set 3 is not resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is not resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 28.48 seconds
Average time: 3.56 seconds


In [74]:
sets = read_sets('Test_Sets/H_4_4.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,4,4)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.186232328414917
0.0997312068939209
0.6630139350891113
0.5495986938476562
0.5407528877258301
0.30783629417419434
0.47206830978393555
0.1403195858001709
Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 2.96 seconds
Average time: 0.37 seconds


In [75]:
sets = read_sets('Test_Sets/H_4_4.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = brute_force_resolve(R,4,4)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.09566712379455566
0.0
0.01521611213684082
0.0371549129486084
0.09752321243286133
0.09847450256347656
0.09369111061096191
0.05012059211730957
Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 0.49 seconds
Average time: 0.06 seconds


In [76]:
sets = read_sets('Test_Sets/H_4_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,5,4)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

4.049058675765991
29.68521499633789
7.734800815582275
5.917059421539307
0.5755825042724609
9.562382698059082
6.755131006240845
0.6377646923065186
Set 1 is not resolving
Set 2 is resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is not resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 64.92 seconds
Average time: 8.11 seconds


In [77]:
sets = read_sets('Test_Sets/H_4_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = brute_force_resolve(R,5,4)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

0.6607890129089355
1.36799955368042
1.3446648120880127
0.033270835876464844
0.03351449966430664
1.2717914581298828
1.2315857410430908
0.06641674041748047
Set 1 is not resolving
Set 2 is resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is not resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 6.01 seconds
Average time: 0.75 seconds


In [78]:
sets = read_sets('Test_Sets/H_4_6.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,6,4)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

30.115206241607666
3.124643087387085
1.7906651496887207
1.9740214347839355
9.459671974182129
2.1499099731445312
2.191436290740967
0.909095048904419
Set 1 is not resolving
Set 2 is not resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is not resolving
Set 7 is resolving
Set 8 is resolving
Total Time: 51.71 seconds
Average time: 6.46 seconds


In [79]:
sets = read_sets('Test_Sets/H_4_6.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = brute_force_resolve(R,6,4)
    is_resolving.append(resolve)
    end = time.time()
    print(end-start)
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

2.225517511367798
1.8270957469940186
21.23843789100647
2.6042275428771973
21.482619285583496
0.5004496574401855
19.88254141807556
20.077540636062622
Set 1 is not resolving
Set 2 is not resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is not resolving
Set 7 is resolving
Set 8 is resolving
Total Time: 89.84 seconds
Average time: 11.23 seconds
